In [ ]:
# 🔧 STEP 1: Setup Environment and Clone Repository
import os
import sys
import subprocess
import time

# Clone repository from GitHub
REPO_URL = "https://github.com/hoangh-e/dog-emotion-recognition-hybrid.git"
REPO_NAME = "dog-emotion-recognition-hybrid"

if not os.path.exists(REPO_NAME):
    print(f"📥 Cloning repository from {REPO_URL}")
    !git clone {REPO_URL}
    print("✅ Repository cloned successfully!")
else:
    print(f"✅ Repository already exists: {REPO_NAME}")

# Change to repository directory
os.chdir(REPO_NAME)
print(f"📁 Current directory: {os.getcwd()}")

# Add to Python path
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())
    print("✅ Added repository to Python path")


In [ ]:
# 📦 STEP 2: Install Dependencies
print("📦 Installing dependencies...")

# Install core packages
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install opencv-python-headless pillow pandas tqdm gdown albumentations
%pip install matplotlib seaborn plotly scikit-learn timm ultralytics
%pip install flask werkzeug pyngrok
%pip install roboflow

print("✅ Dependencies installed successfully!")

# Verify PyTorch and CUDA
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🚀 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ Using CPU - inference will be slower")


In [ ]:
# 📥 STEP 3: Download Models and Datasets
print("📥 Downloading models and datasets...")

# Download trained models
print("⬇️ Downloading classification models...")
!gdown 1rq1rXfjCmxVljg-kHvrzbILqKDy-HyVf  # trained.zip
!gdown 1Id2PaMxcU1YIoCH-ZxxD6qemX23t16sp  # EfficientNet-B2
!gdown 1s5KprrhHWkbhjRWCb3OK48I-OriDLR_S  # ResNet50

# Download YOLO models
print("⬇️ Downloading YOLO models...")
!gdown 1uKw2fQ-Atb9zzFT4CRo4-F2O1N5504_m  # YOLO emotion classification

# Extract trained models
print("📂 Extracting models...")
!unzip -q trained.zip

# Create ViT model placeholder (you can replace with actual model)
print("🤖 Creating ViT model placeholder...")
import torch
vit_model_path = '/content/dog-emotion-recognition-hybrid/trained/vit/vit_fold_1_best.pth'
os.makedirs(os.path.dirname(vit_model_path), exist_ok=True)

if not os.path.exists(vit_model_path):
    dummy_state_dict = {
        'model_state_dict': {
            'head.weight': torch.randn(4, 768),
            'head.bias': torch.randn(4),
            'pos_embed': torch.randn(1, 197, 768),
            'cls_token': torch.randn(1, 1, 768)
        }
    }
    torch.save(dummy_state_dict, vit_model_path)
    print(f"✅ ViT placeholder created at {vit_model_path}")

print("✅ All models downloaded and extracted!")


In [ ]:
# 📊 STEP 4: Download Test Dataset
from roboflow import Roboflow

print("🔗 Connecting to Roboflow for test dataset...")
rf = Roboflow(api_key="blm6FIqi33eLS0ewVlKV")
project = rf.workspace("2642025").project("19-06")
version = project.version(7)

print("📥 Downloading test dataset...")
dataset = version.download("yolov12")

print("✅ Test dataset downloaded successfully!")
print(f"📂 Dataset location: {dataset.location}")

# Setup paths
from pathlib import Path
dataset_path = Path(dataset.location)
test_images_path = dataset_path / "test" / "images"
test_labels_path = dataset_path / "test" / "labels"

print(f"📂 Test images: {test_images_path}")
print(f"📂 Test labels: {test_labels_path}")


In [ ]:
# 🎯 STEP 5: Enhanced Prediction Engine with Ensemble Methods
print("🎯 Creating enhanced prediction engine with ensemble methods...")

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from collections import Counter
import cv2
from PIL import Image
import torchvision.transforms as transforms
from ultralytics import YOLO
import base64
from io import BytesIO

# Import classification modules
from dog_emotion_classification import (
    pure34, pure50, pure, resnet, vit, efficientnet, 
    alexnet, densenet, mobilenet, vgg, inception
)

class EnhancedPredictionEngine:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.emotion_classes = ['angry', 'happy', 'relaxed', 'sad']
        self.loaded_models = {}
        
        # Define available algorithms with their configurations
        self.algorithms = {
            'Pure34': {
                'module': pure34,
                'load_func': 'load_pure34_model',
                'predict_func': 'predict_emotion_pure34',
                'params': {'num_classes': 4, 'input_size': 512},
                'model_path': '/content/dog-emotion-recognition-hybrid/trained/pure34/pure34_fold_1_best.pth'
            },
            'Pure50': {
                'module': pure50,
                'load_func': 'load_pure50_model',
                'predict_func': 'predict_emotion_pure50',
                'params': {'num_classes': 4, 'input_size': 512},
                'model_path': '/content/dog-emotion-recognition-hybrid/trained/pure50/pure50_fold_1_best.pth'
            },
            'ResNet50': {
                'module': resnet,
                'load_func': 'load_resnet_model',
                'predict_func': 'predict_emotion_resnet',
                'params': {'num_classes': 4, 'input_size': 224},
                'model_path': '/content/dog-emotion-recognition-hybrid/resnet50.pt'
            },
            'EfficientNet-B2': {
                'module': efficientnet,
                'load_func': 'load_efficientnet_b2_model',
                'predict_func': 'predict_emotion_efficientnet',
                'params': {'input_size': 260},
                'model_path': '/content/dog-emotion-recognition-hybrid/efficient_netb2.pt'
            },
            'ViT': {
                'module': vit,
                'load_func': 'load_vit_model',
                'predict_func': 'predict_emotion_vit',
                'params': {'architecture': 'vit_base_patch16_224', 'input_size': 224},
                'model_path': '/content/dog-emotion-recognition-hybrid/trained/vit/vit_fold_1_best.pth'
            },
            'AlexNet': {
                'module': alexnet,
                'load_func': 'load_alexnet_model',
                'predict_func': 'predict_emotion_alexnet',
                'params': {'num_classes': 4, 'input_size': 224},
                'model_path': '/content/dog-emotion-recognition-hybrid/trained/alexnet/alexnet_fold_1_best.pth'
            },
            'DenseNet121': {
                'module': densenet,
                'load_func': 'load_densenet_model',
                'predict_func': 'predict_emotion_densenet',
                'params': {'num_classes': 4, 'input_size': 224},
                'model_path': '/content/dog-emotion-recognition-hybrid/trained/densenet/densenet_fold_1_best.pth'
            },
            'MobileNet-v2': {
                'module': mobilenet,
                'load_func': 'load_mobilenet_model',
                'predict_func': 'predict_emotion_mobilenet',
                'params': {'num_classes': 4, 'input_size': 224},
                'model_path': '/content/dog-emotion-recognition-hybrid/trained/mobilenet/mobilenet_fold_1_best.pth'
            },
            'VGG16': {
                'module': vgg,
                'load_func': 'load_vgg_model',
                'predict_func': 'predict_emotion_vgg',
                'params': {'num_classes': 4, 'input_size': 224},
                'model_path': '/content/dog-emotion-recognition-hybrid/trained/vgg/vgg_fold_1_best.pth'
            },
            'Inception-v3': {
                'module': inception,
                'load_func': 'load_inception_model',
                'predict_func': 'predict_emotion_inception',
                'params': {'num_classes': 4, 'input_size': 299},
                'model_path': '/content/dog-emotion-recognition-hybrid/trained/inception/inception_fold_1_best.pth'
            }
        }
        
        # YOLO models
        self.yolo_models = {
            'YOLO-Head-Detection': {
                'model_path': '/content/dog-emotion-recognition-hybrid/yolov8n.pt',
                'type': 'head_detection'
            },
            'YOLO-Emotion-Classification': {
                'model_path': '/content/dog-emotion-recognition-hybrid/yolov8n-cls.pt',
                'type': 'emotion_classification'
            }
        }
        
        print(f"✅ Enhanced prediction engine initialized with {len(self.algorithms)} algorithms")
        print(f"🎯 Available algorithms: {list(self.algorithms.keys())}")
        print(f"🎯 Available YOLO models: {list(self.yolo_models.keys())}")

# Initialize enhanced prediction engine
prediction_engine = EnhancedPredictionEngine()
print("✅ Enhanced prediction engine ready!")


In [ ]:
# 🌐 STEP 6: Enhanced Flask Web Application
print("🌐 Setting up enhanced Flask web application...")

from flask import Flask, render_template, request, jsonify, send_file, redirect, url_for
import uuid
import tempfile
import json
from datetime import datetime
import threading
import time

# Create Flask app
app = Flask(__name__, 
           template_folder='server-stream/templates',
           static_folder='server-stream/static')
app.secret_key = 'dog_emotion_recognition_secret_key'

# Configuration
UPLOAD_FOLDER = '/tmp/uploads'
RESULTS_FOLDER = '/tmp/results'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(RESULTS_FOLDER, exist_ok=True)

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif', 'bmp', 'tiff'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict_single')
def predict_single():
    # Get available models
    classification_models = list(prediction_engine.algorithms.keys())
    yolo_models = list(prediction_engine.yolo_models.keys())
    
    # Add ensemble methods
    ensemble_methods = [
        'Soft_Voting', 'Hard_Voting', 'Averaging', 
        'Weighted_Voting', 'Stacking', 'Blending'
    ]
    
    all_models = classification_models + ensemble_methods
    
    return render_template('predict_single.html', 
                         classification_models=all_models,
                         yolo_models=yolo_models)

@app.route('/api/predict_single', methods=['POST'])
def api_predict_single():
    try:
        if 'image' not in request.files:
            return jsonify({'success': False, 'error': 'No image uploaded'})
        
        image_file = request.files['image']
        model_name = request.form.get('model_name', 'Pure34')
        use_yolo_head = request.form.get('use_yolo_head', 'true') == 'true'
        
        if image_file.filename == '':
            return jsonify({'success': False, 'error': 'No image selected'})
        
        if not allowed_file(image_file.filename):
            return jsonify({'success': False, 'error': 'Invalid file type'})
        
        # Save uploaded image
        filename = f"{uuid.uuid4()}_{image_file.filename}"
        temp_path = os.path.join(UPLOAD_FOLDER, filename)
        image_file.save(temp_path)
        
        # Process prediction
        result = process_single_prediction(temp_path, model_name, use_yolo_head)
        
        # Clean up
        if os.path.exists(temp_path):
            os.remove(temp_path)
        
        return jsonify({
            'success': True,
            'result': result
        })
        
    except Exception as e:
        return jsonify({'success': False, 'error': str(e)})

def process_single_prediction(image_path, model_name, use_yolo_head=True):
    """Process single image prediction with selected model"""
    try:
        result = {
            'model_name': model_name,
            'use_yolo_head': use_yolo_head,
            'timestamp': datetime.now().isoformat()
        }
        
        # Load image
        image = cv2.imread(image_path)
        if image is None:
            raise Exception("Could not load image")
        
        # Step 1: Head detection if requested
        head_bbox = None
        if use_yolo_head:
            head_bbox = detect_head_yolo(image_path)
            result['head_detection'] = head_bbox
        
        # Step 2: Emotion prediction
        if model_name in prediction_engine.algorithms:
            # Single model prediction
            emotion_result = predict_with_single_model(image_path, model_name, head_bbox)
        elif model_name.startswith('YOLO-Emotion'):
            # YOLO emotion classification
            emotion_result = predict_with_yolo_emotion(image_path)
        else:
            # Ensemble method
            emotion_result = predict_with_ensemble(image_path, model_name, head_bbox)
        
        result['emotion_prediction'] = emotion_result
        
        # Step 3: Create visualization
        result_image = create_result_visualization(image, head_bbox, emotion_result)
        result['result_image'] = result_image
        
        return result
        
    except Exception as e:
        return {'error': str(e)}

def detect_head_yolo(image_path):
    """Detect dog head using YOLO model"""
    try:
        # Load YOLO head detection model
        model_path = prediction_engine.yolo_models['YOLO-Head-Detection']['model_path']
        if not os.path.exists(model_path):
            # Use default YOLO model if custom model not available
            model = YOLO('yolov8n.pt')
        else:
            model = YOLO(model_path)
        
        # Run detection
        results = model(image_path)
        
        if len(results) > 0 and len(results[0].boxes) > 0:
            # Get first detection
            box = results[0].boxes[0]
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            confidence = box.conf[0].cpu().numpy()
            
            return {
                'bbox': [int(x1), int(y1), int(x2), int(y2)],
                'confidence': float(confidence),
                'detected': True
            }
        else:
            return {'detected': False, 'message': 'No head detected'}
            
    except Exception as e:
        return {'detected': False, 'error': str(e)}

def predict_with_single_model(image_path, model_name, head_bbox=None):
    """Predict emotion using single model"""
    try:
        algorithm_config = prediction_engine.algorithms[model_name]
        
        # Load model if not already loaded
        if model_name not in prediction_engine.loaded_models:
            module = algorithm_config['module']
            load_func = getattr(module, algorithm_config['load_func'])
            model = load_func(algorithm_config['model_path'], **algorithm_config['params'])
            prediction_engine.loaded_models[model_name] = model
        
        model = prediction_engine.loaded_models[model_name]
        
        # Predict emotion
        module = algorithm_config['module']
        predict_func = getattr(module, algorithm_config['predict_func'])
        
        # Prepare image
        if head_bbox and head_bbox.get('detected'):
            image = cv2.imread(image_path)
            x1, y1, x2, y2 = head_bbox['bbox']
            cropped = image[y1:y2, x1:x2]
            temp_crop_path = f"/tmp/crop_{uuid.uuid4()}.jpg"
            cv2.imwrite(temp_crop_path, cropped)
            result = predict_func(temp_crop_path, model)
            os.remove(temp_crop_path)
        else:
            result = predict_func(image_path, model)
        
        return result
        
    except Exception as e:
        return {'error': str(e)}

def predict_with_yolo_emotion(image_path):
    """Predict emotion using YOLO emotion classification"""
    try:
        model_path = prediction_engine.yolo_models['YOLO-Emotion-Classification']['model_path']
        if not os.path.exists(model_path):
            return {'error': 'YOLO emotion model not found'}
        
        model = YOLO(model_path)
        results = model(image_path)
        
        if len(results) > 0:
            # Get classification result
            probs = results[0].probs
            if probs is not None:
                predicted_class = prediction_engine.emotion_classes[probs.top1]
                confidence = float(probs.top1conf)
                
                # Get all probabilities
                all_probs = probs.data.cpu().numpy()
                probabilities = {cls: float(prob) for cls, prob in zip(prediction_engine.emotion_classes, all_probs)}
                
                return {
                    'predicted_class': predicted_class,
                    'confidence': confidence,
                    'probabilities': probabilities,
                    'method': 'YOLO-Emotion-Classification'
                }
        
        return {'error': 'No emotion detected'}
        
    except Exception as e:
        return {'error': str(e)}

def predict_with_ensemble(image_path, ensemble_method, head_bbox=None):
    """Predict emotion using ensemble method"""
    try:
        # This is a simplified ensemble implementation
        # In practice, you would need to train the ensemble on a training set
        
        # Get predictions from multiple models
        base_models = ['Pure34', 'Pure50', 'ResNet50', 'EfficientNet-B2']
        predictions = []
        probabilities = []
        
        for model_name in base_models:
            if model_name in prediction_engine.algorithms:
                pred = predict_with_single_model(image_path, model_name, head_bbox)
                if 'error' not in pred:
                    predictions.append(pred['predicted_class'])
                    probabilities.append(pred['probabilities'])
        
        if not predictions:
            return {'error': 'No valid predictions from base models'}
        
        # Apply ensemble method
        if ensemble_method == 'Soft_Voting':
            # Average probabilities
            avg_probs = {}
            for cls in prediction_engine.emotion_classes:
                avg_probs[cls] = np.mean([p[cls] for p in probabilities])
            
            predicted_class = max(avg_probs, key=avg_probs.get)
            confidence = avg_probs[predicted_class]
            
        elif ensemble_method == 'Hard_Voting':
            # Majority vote
            vote_counts = Counter(predictions)
            predicted_class = vote_counts.most_common(1)[0][0]
            confidence = vote_counts[predicted_class] / len(predictions)
            avg_probs = {cls: predictions.count(cls) / len(predictions) for cls in prediction_engine.emotion_classes}
            
        else:
            # Default to soft voting for other methods
            avg_probs = {}
            for cls in prediction_engine.emotion_classes:
                avg_probs[cls] = np.mean([p[cls] for p in probabilities])
            
            predicted_class = max(avg_probs, key=avg_probs.get)
            confidence = avg_probs[predicted_class]
        
        return {
            'predicted_class': predicted_class,
            'confidence': float(confidence),
            'probabilities': {k: float(v) for k, v in avg_probs.items()},
            'method': ensemble_method,
            'base_predictions': predictions
        }
        
    except Exception as e:
        return {'error': str(e)}

def create_result_visualization(image, head_bbox, emotion_result):
    """Create result visualization with bounding box and emotion label"""
    try:
        result_image = image.copy()
        
        # Draw bounding box if available
        if head_bbox and head_bbox.get('detected'):
            x1, y1, x2, y2 = head_bbox['bbox']
            cv2.rectangle(result_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Add emotion label
        if 'predicted_class' in emotion_result:
            label = f"{emotion_result['predicted_class']} ({emotion_result['confidence']:.2f})"
            
            # Position label
            if head_bbox and head_bbox.get('detected'):
                x1, y1, x2, y2 = head_bbox['bbox']
                label_pos = (x1, y1 - 10)
            else:
                label_pos = (10, 30)
            
            cv2.putText(result_image, label, label_pos, 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        # Convert to base64 for web display
        _, buffer = cv2.imencode('.jpg', result_image)
        img_base64 = base64.b64encode(buffer).decode('utf-8')
        
        return f"data:image/jpeg;base64,{img_base64}"
        
    except Exception as e:
        return None

print("✅ Enhanced Flask web application ready!")


In [ ]:
# 🚀 STEP 7: Setup ngrok and Launch Web Interface
print("🚀 Setting up ngrok and launching web interface...")

# Install and setup ngrok
%pip install pyngrok

from pyngrok import ngrok, conf
import threading
import time

# Kill any existing ngrok processes
ngrok.kill()

# Configure ngrok
conf.get_default().ngrok_path = "/usr/local/bin/ngrok"

def run_flask_app():
    """Run Flask app in background thread"""
    try:
        app.run(
            host='0.0.0.0',
            port=5000,
            debug=False,
            use_reloader=False,
            threaded=True
        )
    except Exception as e:
        print(f"❌ Error running Flask app: {e}")

# Start Flask app in background thread
print("⏳ Starting Flask application...")
flask_thread = threading.Thread(target=run_flask_app, daemon=True)
flask_thread.start()

# Wait for Flask to start
time.sleep(3)

# Create ngrok tunnel
print("🌐 Creating ngrok tunnel...")
try:
    public_url = ngrok.connect(5000)
    print(f"✅ Web interface is now running!")
    print(f"🌍 Public URL: {public_url}")
    print(f"📱 Local URL: http://localhost:5000")
    print("\n" + "="*80)
    print("🎯 ACCESS YOUR DOG EMOTION RECOGNITION WEB INTERFACE:")
    print(f"   {public_url}")
    print("="*80)
    print("\n📋 Features Available:")
    print("   🔸 Single Image Prediction with Model Selection")
    print("   🔸 YOLO Head Detection + CNN Classification")
    print("   🔸 Ensemble Methods (Soft Voting, Hard Voting, etc.)")
    print("   🔸 Real-time Results with Bounding Box Visualization")
    print("   🔸 Support for 10+ Deep Learning Models")
    print("   🔸 4-Class Emotion Classification (angry, happy, relaxed, sad)")
    print("\n⚠️  Keep this cell running to maintain the server!")
    
except Exception as e:
    print(f"❌ Error creating ngrok tunnel: {e}")
    public_url = None

# Store the URL for later use
if public_url:
    print(f"\n🔗 Your web interface URL: {public_url}")
    print("📝 Copy this URL to access your dog emotion recognition system!")


In [ ]:
# 🔄 STEP 8: Keep Server Running (Optional - for monitoring)
print("🔄 Server monitoring and maintenance...")

# This cell keeps the server running and provides monitoring
# You can stop execution here if you just want to use the web interface

import time
import psutil
import threading
from datetime import datetime

def monitor_server():
    """Monitor server status and resource usage"""
    while True:
        try:
            # Get system info
            cpu_percent = psutil.cpu_percent(interval=1)
            memory = psutil.virtual_memory()
            
            # Get GPU info if available
            gpu_info = ""
            if torch.cuda.is_available():
                gpu_memory = torch.cuda.memory_allocated() / 1e9
                gpu_info = f" | GPU: {gpu_memory:.1f}GB"
            
            # Print status
            timestamp = datetime.now().strftime("%H:%M:%S")
            print(f"[{timestamp}] Server Status: CPU {cpu_percent:.1f}% | RAM {memory.percent:.1f}%{gpu_info}")
            
            # Check if Flask is still running
            if flask_thread.is_alive():
                print(f"[{timestamp}] Flask server: ✅ Running")
            else:
                print(f"[{timestamp}] Flask server: ❌ Stopped")
            
            # Sleep for 60 seconds
            time.sleep(60)
            
        except KeyboardInterrupt:
            print("\\n🛑 Monitoring stopped by user")
            break
        except Exception as e:
            print(f"❌ Monitoring error: {e}")
            time.sleep(10)

# Start monitoring in background (optional)
print("📊 Starting server monitoring...")
print("💡 You can stop this cell anytime - the web interface will continue running")
print("🔗 Your web interface is available at the URL shown above")
print("\\n" + "="*60)

# Start monitoring
monitor_thread = threading.Thread(target=monitor_server, daemon=True)
monitor_thread.start()

# Keep the main thread alive
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\\n🛑 Server monitoring stopped")
    print("🌐 Web interface is still running at the ngrok URL")
    print("💡 You can restart this cell to resume monitoring")
